# Metrics evaluation notebook

_Alex Malz (NYU)_
_Renee Hlozek (Toronto)_
_Rahul Biswas (Stockholm University)_
_Tarek Alam (UCL)_
_Rafael Martinez-Galarza (Harvard)_
_Anita Bahmanyar (Toronto)_


In [1]:
# import string
# import itertools
# import random
# import os
# import csv

import numpy as np
import pandas as pd

import proclam
from proclam import *

In [2]:
import matplotlib as mpl
mpl.use('Agg')
mpl.rcParams['text.usetex'] = False
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['savefig.dpi'] = 250
mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['savefig.bbox'] = 'tight'
import matplotlib.pyplot as plt
%matplotlib inline

This notebook defines the metrics (and corresponding truth tables) to be run via an input file and to produce plots based on the output.

In [3]:
metricslist = ['Brier', 'LogLoss']
colors = ['b', 'r']
dirname = 'examples/'
markerlist = ['o', 's', '*']

## Read in data

Loading in the data from the Mystery Dataset

In [4]:
mystery = {}
mystery['label'] = 'Unknown'
mystery['names'] = ['RandomForest', 'KNeighbors', 'MLPNeuralNet']

Loading in the data from SNPhotCC

In [5]:
snphotcc = {}
snphotcc['label'] = 'SNPhotCC'
prefixes = ['Templates', 'Wavelets']
suffixes = ['BoostForest', 'KNN', 'NB', 'NeuralNetwork', 'SVM']
snphotcc['names'] = []
for prefix in prefixes:
    for suffix in suffixes:
        snphotcc['names'].append(prefix+suffix)

Loading in the data from ProClaM

In [6]:
plasticc = {}
plasticc['label'] = 'ProClaM'
plasticc['names'] = ['Idealized', 'Guess', 'Tunnel', 'Broadbrush', 'Cruise', 'SubsumedTo', 'SubsumedFrom']

In [7]:
# more_names = snphotcc_names
# more_classifications = ['%s/predicted_prob_%s.csv'%(name,name) for name in snphotcc_names]
# more_truth_tables = ['%s/truth_table_%s.csv'%(name,name) for name in snphotcc_names]
# more_class_pairs = zip(more_classifications, more_truth_tables)

In [8]:
def make_class_pairs(data_info_dict):
    return zip(data_info_dict['classifications'], data_info_dict['truth_tables'])

def make_file_locs(data_info_dict):
    names = data_info_dict['names']
    data_info_dict['dirname'] = dirname + data_info_dict['label'] + '/'
    data_info_dict['classifications'] = ['%s/predicted_prob_%s.csv'%(name, name) for name in names]
    data_info_dict['truth_tables'] = ['%s/truth_table_%s.csv'%(name, name) for name in names]
    print(data_info_dict)
    return data_info_dict

In [9]:
for dataset in [ plasticc]: #mystery, snphotcc,
    dataset = make_file_locs(dataset)
    dataset['class_pairs'] = make_class_pairs(dataset)

{'label': 'ProClaM', 'names': ['Idealized', 'Guess', 'Tunnel', 'Broadbrush', 'Cruise', 'SubsumedTo', 'SubsumedFrom'], 'dirname': 'examples/ProClaM/', 'classifications': ['Idealized/predicted_prob_Idealized.csv', 'Guess/predicted_prob_Guess.csv', 'Tunnel/predicted_prob_Tunnel.csv', 'Broadbrush/predicted_prob_Broadbrush.csv', 'Cruise/predicted_prob_Cruise.csv', 'SubsumedTo/predicted_prob_SubsumedTo.csv', 'SubsumedFrom/predicted_prob_SubsumedFrom.csv'], 'truth_tables': ['Idealized/truth_table_Idealized.csv', 'Guess/truth_table_Guess.csv', 'Tunnel/truth_table_Tunnel.csv', 'Broadbrush/truth_table_Broadbrush.csv', 'Cruise/truth_table_Cruise.csv', 'SubsumedTo/truth_table_SubsumedTo.csv', 'SubsumedFrom/truth_table_SubsumedFrom.csv']}


## Make the plots

In [10]:
def plot_cm(probs, truth, name, loc=''):
    cm = proclam.metrics.util.prob_to_cm(probs, truth)
    plt.matshow(cm.T, vmin=0., vmax=1.)
# plt.xticks(range(max(truth)+1), names)
# plt.yticks(range(max(truth)+1), names)
    plt.xlabel('predicted class')
    plt.ylabel('true class')
    plt.colorbar()
    plt.title(name)
    #plt.savefig(loc+name+'_cm.png')
    plt.show()
    #plt.close()

In [11]:
def read_class_pairs(pair, dataset, cc):#loc='', title=''):
    loc=dataset['dirname']
    title=dataset['label']+' '+dataset['names'][cc]
    clfile = pair[0]
    truthfile = pair[1]
    prob_mat = pd.read_csv(loc+clfile, delim_whitespace=True).values
    nobj = np.shape(prob_mat)[0]
    nclass = np.shape(prob_mat)[1]
    truth_values = pd.read_csv(loc+truthfile, delim_whitespace=True).values
    nobj_truth = np.shape(truth_values)[0]
    nclass_truth = np.shape(truth_values)[1]
    tvec = np.where(truth_values==1)[1]
    print(tvec)
#     if nclass_truth!= nclass:
#         print('Truth table of size %i x %i and prob matrix of size %i x %i do not match up in size'%(nobj,nclass,nobj_truth,nclass_truth))
#     else:
#         print('Considering classifications with %i classes'%nclass)
    pmat = prob_mat
    plot_cm(pmat, tvec, title, loc=loc+dataset['names'][cc]+'/')
    return pmat, tvec

In [12]:
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)
        
def per_metric_helper(ax, n, data, metric_names, codes, shapes, colors):
    plot_n = n+1
    in_x = np.arange(len(codes))
    ax_n = ax
    n_factor = 0.1 * (plot_n - 2)
    if plot_n>1:
        ax_n = ax.twinx()
        rot_ang = 270
        label_space = 15.
    else:
        rot_ang = 90
        label_space = 0.
    if plot_n>2:
        ax_n.spines["right"].set_position(("axes", 1. + 0.1 * (plot_n-1)))
        make_patch_spines_invisible(ax_n)
        ax_n.spines["right"].set_visible(True)
    handle = ax_n.scatter(in_x+n_factor*np.ones_like(data[n]), data[n], marker=shapes[n], s=10, color=colors[n], label=metric_names[n])
    ax_n.set_ylabel(metric_names[n], rotation=rot_ang, fontsize=14, labelpad=label_space)
#     ax_n.set_ylim(0.9 * min(data[n]), 1.1 * max(data[n]))
    return(ax, ax_n, handle)

def metric_plot(dataset, metric_names, shapes, colors):
    codes = dataset['names']
    data = dataset['results']
    title = dataset['label']
    fileloc = dataset['dirname']+dataset['label']+'_results.png'
    xs = np.arange(len(codes))
    fig, ax = plt.subplots()
    fig.subplots_adjust(right=1.)
    handles = []
    for n in range(len(metric_names)):
        (ax, ax_n, handle) = per_metric_helper(ax, n, data, metric_names, codes, shapes, colors)
        handles.append(handle)
    plt.xticks(xs, codes)
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
    plt.xlabel('Classifiers', fontsize=14)
    plt.legend(handles, metric_names)
    plt.suptitle(title)
    plt.savefig(fileloc)
    plt.show()
    return

make a plot of classifier on the x-axis and metric score on the y-axis


In [13]:
for dataset in [mystery, snphotcc, plasticc]:
    data = np.empty((len(metricslist), len(dataset['names'])))
    for cc, pair in enumerate(dataset['class_pairs']):
        print(pair)
        probm, truthv = read_class_pairs(pair, dataset, cc)#loc=dataset['dirname'], title=dataset['label']+' '+dataset['names'][cc])
        for count, metric in enumerate(metricslist):
            D = getattr(proclam.metrics, metric)()
            hm = D.evaluate(probm, truthv)
            data[count][cc] = hm
    dataset['results'] = data
    metric_plot(dataset, metricslist, markerlist, colors)

KeyError: 'class_pairs'